In [18]:
import math as ma

In [68]:
import pandas as pd
data=pd.read_csv('data.csv')
dd=data.sample(n=4000,random_state=1,axis=0)

In [69]:
data.index

RangeIndex(start=0, stop=6272, step=1)

# coverage in validation set

In [70]:
for i in dd.index:
    data=data.drop(index=i,axis=1)

In [71]:
ab=data['Abstract'].values.tolist()  #.drop_duplicates()

In [92]:
ava=pd.read_csv('a0_v_a1_result.csv')
va=pd.read_csv('v_a1_result.csv')
av=pd.read_csv(('a1_v_result.csv'))

In [93]:
ava=ava[['v','n','hyx']].values.tolist()

In [94]:
va=va[['v','n','hyx']].values.tolist()
av=av[['v','n','hyx']].values.tolist()

In [95]:
#total=[]
#for i in ava:
#    total.append(i)
#for i in va:
#    total.append(i)
#for i in av:
#    total.append(i)

In [96]:
total1=[]
total2=[]
total3=[]
def get_hyx_model(total1,total2,total3,ava,av,va,n):
    for i in ava:
        if i[2]>0:
            total1.append(i)
    for i in va:
        if i[2]>0:
            total2.append(i)
    for i in av:
        if i[2]>0:
            total3.append(i)
    total1=pd.DataFrame(total1,columns=['v','n','hyx'])   #get a0-v-a1 model
    total2=pd.DataFrame(total2,columns=['v','n','hyx'])   #get v-a1 model
    total3=pd.DataFrame(total3,columns=['v','n','hyx'])   #get a1-v model
    total1=total1.groupby('v').head(n).reset_index(drop=True)  #get n head of each v_n group
    total2=total2.groupby('v').head(n).reset_index(drop=True)
    total3=total3.groupby('v').head(n).reset_index(drop=True)
    total1=total1.values.tolist()
    total2=total2.values.tolist()
    total3=total3.values.tolist()
    #total=pd.concat([total1,total2,total3],axis=0)   #combine model
    #total=total.drop_duplicates()   
    #total=total.values.tolist()

In [ ]:
def get_rep_model(total1,total2,total3,ava,av,va,n):
    for i in ava:
        #if i[2]>0:
        total1.append(i)
    for i in va:
        #if i[2]>0:
        total2.append(i)
    for i in av:
        #if i[2]>0:
        total3.append(i)
    total1=pd.DataFrame(total1,columns=['v','n','rep'])   
    total2=pd.DataFrame(total2,columns=['v','n','rep'])   
    total3=pd.DataFrame(total3,columns=['v','n','rep'])   
    total1=total1.groupby('v').head(n).reset_index(drop=True)  
    total2=total2.groupby('v').head(n).reset_index(drop=True)
    total3=total3.groupby('v').head(n).reset_index(drop=True)
    total1=total1.values.tolist()
    total2=total2.values.tolist()
    total3=total3.values.tolist()
    #total=pd.concat([total1,total2,total3],axis=0)   
    #total=total.drop_duplicates()   
    #total=total.values.tolist()

In [ ]:
get_hyx_model(total1,total2,total3,ava,va,av,3)
get_rep_model(total1,total2,total3,ava,va,av,3)

In [77]:
def getA(seg,i,j):
    s=''
    if i==j:
        s=seg[0][i]
    else:
        while i<=j:
            s=s+seg[0][i]
            i=i+1
    return s

In [10]:
from ltp import LTP
ltp = LTP()
ltp.add_words(words=["人大代表", "生态文明","扬子江","长江大保护","习近平",'禁捕','退捕','禁渔'], max_window=5)

In [54]:
def matchw(zz,dict1,dict2,dict3,f):
    for k in zz:
            k=k.replace(' ','')
            k=k.replace('[','（')
            k=k.replace(']','）')
            k=k.replace('(','（')
            k=k.replace(')','）')
            k=k.replace('^','.')
            try:
                seg,hidden=ltp.seg([k])
            except Exception:
                print(k)
                print(zz)
            srl = ltp.srl(hidden, keep_empty=False)
            tt=0                
            start=0
            end=0
            e=0
            for i in range(len(srl[0])-1,-1,-1):
                f1=0
                f2=0
                b=''
                c=''
                d=''
                if len(seg[0][srl[0][i][0]])>1:
                    for j in srl[0][i][1]:
                        #print(j)
                        if tt==0:
                            if j[0]=='A0':
                                a1=getA(seg,j[1],j[2])
                                f1=1
                                end=j[2]
                            elif j[0]=='A1':
                                b=getA(seg,j[1],j[2])
                                if j[1]>srl[0][i][0]:
                                    c=b
                                elif j[1]<srl[0][i][0]:
                                    d=b
                                start=j[1]
                                f2=1
                                tt=1
                            elif j[0]=='A2':
                                if b=='':
                                    b=getA(seg,j[1],j[2])
                                    if j[1]>srl[0][i][0]:
                                        c=b
                                    elif j[1]<srl[0][i][0]:
                                        d=b
                                start=j[1]
                                f2=1
                                tt=1
                        else:
                            if j[0]=='A0':
                                a1=getA(seg,j[1],j[2])
                                f1=1
                                end=j[2]
                            elif j[0]=='A1' and j[2]<start :
                                b=getA(seg,j[1],j[2])
                                #print(b)
                                if j[1]>srl[0][i][0]:
                                    c=b
                                elif j[1]<srl[0][i][0]:
                                    d=b
                                    #print(d)
                                start=j[1]
                                f2=1
                            elif j[0]=='A2' and j[2]<start :
                                if b=='':
                                    b=getA(seg,j[1],j[2])
                                    if j[1]>srl[0][i][0]:
                                        c=b
                                    elif j[1]<srl[0][i][0]:
                                        d=b
                                start=j[1]
                                f2=1
                    #print('start')
                    #print(start)
                    #print('end')
                    #print(end)
                    #print('start')
                    #print(start)
                    #print('end')
                    #print(end)
                    if f1==1 and f2==1 and end<=start and len(seg[0][srl[0][i][0]])==2:
                        if (a1,seg[0][srl[0][i][0]],b) not in dict1.keys() and (a1,seg[0][srl[0][i][0]],b) not in f:
                            dict1[(a1,seg[0][srl[0][i][0]],b)]=1
                            f.append((a1,seg[0][srl[0][i][0]],b))
                        else:
                            if (a1,seg[0][srl[0][i][0]],b) not in f:
                                dict1[(a1,seg[0][srl[0][i][0]],b)]=dict1[(a1,seg[0][srl[0][i][0]],b)]+1
                                f.append((a1,seg[0][srl[0][i][0]],b))
                    elif f1==0 and f2==1 and c!='' and len(seg[0][srl[0][i][0]])==2:
                        if (seg[0][srl[0][i][0]],c) not in dict2.keys() and (seg[0][srl[0][i][0]],c) not in f:
                            dict2[(seg[0][srl[0][i][0]],c)]=1
                            f.append((seg[0][srl[0][i][0]],c))
                        else:
                            if (seg[0][srl[0][i][0]],c) not in f:
                                dict2[(seg[0][srl[0][i][0]],c)]=dict2[(seg[0][srl[0][i][0]],c)]+1
                                f.append((seg[0][srl[0][i][0]],c))
                    elif f1==0 and f2==1 and d!='' and len(seg[0][srl[0][i][0]])==2:
                        if (seg[0][srl[0][i][0]],d) not in dict3.keys() and (seg[0][srl[0][i][0]],d) not in f:
                            dict3[(seg[0][srl[0][i][0]],d)]=1
                            f.append((seg[0][srl[0][i][0]],d))
                        else:
                            if (seg[0][srl[0][i][0]],d) not in f:
                                dict3[(seg[0][srl[0][i][0]],d)]=dict3[(seg[0][srl[0][i][0]],d)]+1
                                f.append((seg[0][srl[0][i][0]],d))

In [78]:
dict1={}
dict2={}
dict3={}
for z in ab:
    f=[]
    z=z.replace('\u200b','')
    z=z.replace('.','^')
    zz=ltp.sent_split([z])
    matchw(zz,dict1,dict2,dict3,f)
    g=z.replace('，','。')
    xx=ltp.sent_split([g])
    matchw(xx,dict1,dict2,dict3,f)

In [79]:
a0a0=[]
for i in dict1.keys():
    a0a0.append([i[0],i[1],i[2],dict1[i]])
vva1=[]
for i in dict2.keys():
    vva1.append([i[0],i[1],dict2[i]])
a1a1v=[]
for i in dict3.keys():
    a1a1v.append([i[1],i[0],dict3[i]])

In [97]:

m0=[]
for i in a0a0:
    for j in total1:
        if i[1]==j[0] and j[1] in i[2]:
            if [j[0],j[1]] not in m0:
                m0.append([j[0],j[1]])
            break
print('a0-v-a1条数')
print(len(total1))
print('匹配上的条数')
print(len(m0))

m1=[]
for i in vva1:
    for j in total2:
        if i[0]==j[0] and j[1] in i[1]:
            if [j[0],j[1]] not in m1:
                m1.append([j[0],j[1]])
            break
print('v-a1条数')
print(len(total2))
print('匹配上的条数')
print(len(m1))
m2=[]
for i in a1a1v:
    for j in total3:
        if i[1]==j[0] and j[1] in i[0]:
            if [j[0],j[1]] not in m2:
                m2.append([j[0],j[1]])
            break
print('a1-v条数')
print(len(total3))
print('匹配上的条数')
print(len(m2))

a0-v-a1条数
488
匹配上的条数
199
v-a1条数
538
匹配上的条数
246
a1-v条数
181
匹配上的条数
65


# coverage in test set

In [37]:
import pandas as pd
test4=pd.read_csv('changjiang_test_1000.csv')

In [39]:
#test=pd.concat([test,test1,test2],axis=0)

In [40]:
#test=test.drop_duplicates().sample(n=1500,random_state=1)
#test4=test4.drop_duplicates()

In [41]:
test4=test4[['摘要']].drop_duplicates()

In [42]:
ab=test4['摘要'].values.tolist()

In [43]:
def matchs(zz):
    r=[]
    for k in zz:
            k=k.replace(' ','')
            k=k.replace('[','（')
            k=k.replace(']','）')
            k=k.replace('(','（')
            k=k.replace(')','）')
            k=k.replace('^','.')
            try:
                seg,hidden=ltp.seg([k])
            except Exception:
                print(k)
                print(zz)
            srl = ltp.srl(hidden, keep_empty=False)
            tt=0                
            start=0
            end=0
            e=0
            for i in range(len(srl[0])-1,-1,-1):
                f1=0
                f2=0
                b=''
                c=''
                d=''
                if len(seg[0][srl[0][i][0]])==2:
                    for j in srl[0][i][1]:
                        #print(j)
                        if tt==0:
                            if j[0]=='A0':
                                a1=getA(seg,j[1],j[2])
                                f1=1
                                end=j[2]
                            elif j[0]=='A1':
                                b=getA(seg,j[1],j[2])
                                if j[1]>srl[0][i][0]:
                                    c=b
                                elif j[1]<srl[0][i][0]:
                                    d=b
                                start=j[1]
                                f2=1
                                tt=1
                            elif j[0]=='A2':
                                if b=='':
                                    b=getA(seg,j[1],j[2])
                                    if j[1]>srl[0][i][0]:
                                        c=b
                                    elif j[1]<srl[0][i][0]:
                                        d=b
                                start=j[1]
                                f2=1
                                tt=1
                        else:
                            if j[0]=='A0':
                                a1=getA(seg,j[1],j[2])
                                f1=1
                                end=j[2]
                            elif j[0]=='A1' and j[2]<start :
                                b=getA(seg,j[1],j[2])
                                #print(b)
                                if j[1]>srl[0][i][0]:
                                    c=b
                                elif j[1]<srl[0][i][0]:
                                    d=b
                                    #print(d)
                                start=j[1]
                                f2=1
                            elif j[0]=='A2' and j[2]<start :
                                if b=='':
                                    b=getA(seg,j[1],j[2])
                                    if j[1]>srl[0][i][0]:
                                        c=b
                                    elif j[1]<srl[0][i][0]:
                                        d=b
                                start=j[1]
                                f2=1
                    #print('start')
                    #print(start)
                    #print('end')
                    #print(end)
                    if f1==1 and f2==1 and end<=start:
                        if [seg[0][srl[0][i][0]],b] not in r:
                            r.append([seg[0][srl[0][i][0]],b])
                    elif f1==0 and f2==1 and c!='':
                        if [seg[0][srl[0][i][0]],c] not in r:
                            r.append([seg[0][srl[0][i][0]],c])
                    elif f1==0 and f2==1 and d!='':
                        if [seg[0][srl[0][i][0]],d] not in r:
                            r.append([seg[0][srl[0][i][0]],d])

    return r

In [44]:
n=1
for i in range(1,10):
    t=0
    y=0
    u=0
    total1=[]
    total2=[]
    total3=[]
    for i in ava:
        if i[2]>0:
            total1.append(i)
    for i in va:
        if i[2]>0:
            total2.append(i)
    for i in av:
        if i[2]>0:
            total3.append(i)
    total1=pd.DataFrame(total1,columns=['v','n','hyx'])   #get a0-v-a1 model
    total2=pd.DataFrame(total2,columns=['v','n','hyx'])   #get v-a1 model
    total3=pd.DataFrame(total3,columns=['v','n','hyx'])   #get a1-v model
    total1=total1.groupby('v').head(n).reset_index(drop=True)  #get n head of each v_n group
    total2=total2.groupby('v').head(n).reset_index(drop=True)
    total3=total3.groupby('v').head(n).reset_index(drop=True)
    total=pd.concat([total1,total2,total3],axis=0)   #combine model
    total=total.drop_duplicates()   #drop duplicates
    #total=total.groupby('v').apply(lambda x: x.sort_values('rep', ascending=False)).reset_index(drop=True).groupby('v').head(n).reset_index(drop=True)
    total=total.values.tolist()
    count=0
    m=[]
    for z in ab:
        z=z.replace('\u200b','')
        z=z.replace('.','$$$')
        zz=ltp.sent_split([z])    
        r=matchs(zz)         #get news a-v-a
        g=z.replace('，','。') 
        xx=ltp.sent_split([g])
        x=matchs(xx)         #get news a-v-a twice
        for l in x:
            if l not in r:
                r.append(l)   #combine
        f1=0
        nn=[]
        for i in r:
            for j in total:    ##match
                if i[0]==j[0] and j[1] in i[1]:
                    f1=1
                    if [j[0],j[1]] not in nn:
                        nn.append([j[0],j[1]])
                        break
        z=z.replace('$$$','.')             
        if f1==1:
            count=count+1
        if len(r)==0:
            m.append([z,0])
        else:
            m.append([z,len(nn)/len(r)])
    pd.DataFrame(m,columns=['s','r']).to_csv('test_hyx_5.12_coverage_'+str(n)+'.csv',index=False,encoding='utf_8_sig')
    print('done')
    print(count/len(ab))
    n=n+1

done
0.593
done
0.859
done
0.913
done
0.94
done
0.949
done
0.973


KeyboardInterrupt: 